## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-13-00-00 +0000,missionlocal,Lost limb,https://missionlocal.org/2025/08/sf-photos-los...
1,2025-08-19-12-56-44 +0000,nyt,Live Updates: European Leaders Discuss Protect...,https://www.nytimes.com/live/2025/08/19/us/tru...
2,2025-08-19-12-54-12 +0000,wapo,Key moments from Trump and Zelensky’s second W...,https://www.washingtonpost.com/politics/2025/0...
3,2025-08-19-12-54-03 +0000,bbc,Air Canada to resume flights after pay deal st...,https://www.bbc.com/news/articles/cewyzg8rpxpo...
4,2025-08-19-12-52-00 +0000,wsj,Housing Starts Climbed Unexpectedly in July,https://www.wsj.com/economy/housing/housing-st...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,53
8,ukraine,25
15,zelensky,23
253,will,16
18,canada,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
230,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...,158
122,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,140
52,2025-08-19-10-25-29 +0000,nypost,Putin and Zelensky could meet within next two ...,https://nypost.com/2025/08/19/world-news/putin...,125
69,2025-08-19-09-30-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,124
259,2025-08-18-16-05-04 +0000,nyt,Trump Will Likely Press Zelensky to Make Conce...,https://www.nytimes.com/2025/08/18/us/zelensky...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
230,158,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...
282,90,2025-08-18-13-53-28 +0000,cbc,Air Canada flight attendants' union says it wi...,https://www.cbc.ca/news/politics/air-canada-st...
115,64,2025-08-19-04-24-10 +0000,nypost,White House mocks ‘unserious’ NJ Dem Rep. LaMo...,https://nypost.com/2025/08/19/us-news/white-ho...
31,46,2025-08-19-11-04-00 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
149,44,2025-08-19-00-22-32 +0000,nypost,Furious Harvard researcher claims he was pumme...,https://nypost.com/2025/08/18/us-news/furious-...
239,37,2025-08-18-17-26-27 +0000,nyt,Did You Recently Buy a Used E.V.? We Want to H...,https://www.nytimes.com/2025/08/18/business/us...
209,36,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
124,34,2025-08-19-02-27-43 +0000,nypost,Elon Musk baby mama Ashley St. Clair reveals n...,https://nypost.com/2025/08/18/us-news/elon-mus...
274,32,2025-08-18-15-03-21 +0000,wapo,Newsmax to pay $67 million to settle Dominion ...,https://www.washingtonpost.com/business/2025/0...
175,32,2025-08-18-22-20-19 +0000,nypost,MSNBC gets mercilessly mocked over new MS NOW ...,https://nypost.com/2025/08/18/media/msnbc-gets...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
